In [328]:
!pip install pyspark

In [329]:
!pip install neo4j

In [330]:
import neo4j
from neo4j import GraphDatabase
import pandas as pd

from itertools import product

In [331]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .getOrCreate()

sc = spark.sparkContext
sc

<SparkContext master=local[*] appName=pyspark-shell>

Grafo de prueba

In [332]:
n1 = [1, 1, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5]
label = [11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12, 12]
n2 = [2, 3, 3, 2, 4, 1, 2, 3, 5, 1, 2, 6]

df = pd.DataFrame(data={'n1' : n1, 'label' : label, 'n2' : n2})
df

,n1,label,n2
0,1,11,2
1,1,11,3
2,2,11,3
3,3,11,2
4,3,11,4
5,4,11,1
6,4,11,2
7,4,11,3
8,4,12,5
9,5,12,1


P1 - Implementar Neo4JS

Nos conectamos a una base de Neo4j

In [333]:
URI = "neo4j+s://cc330f29.databases.neo4j.io"
AUTH = ("neo4j","UXROJ0E-Lv4ncdteIU_5hq4gBHY3sn7uSvQpUJQCs80")

driver = GraphDatabase.driver(URI, auth=AUTH)
with driver.session() as session:
    try:
        session.run("RETURN 1")
        print("Connection to Neo4j established successfully!")
    except Exception as e:
        print(f"Failed to connect to Neo4j: {e}")


Connection to Neo4j established successfully!


In [334]:
def create_relationships(session, info):

    query = """
    UNWIND $edges AS row
    MERGE (n1:Node {id:row.n1})
    MERGE (n2:Node {id:row.n2})
    MERGE (n1)-[r:Label {value:row.label}]->(n2);
    """
    session.run(query, edges=info)


Cargamos los datos y los leemos desde la base

In [335]:

# Insert data into Neo4j
with driver.session() as session:
  create_relationships(session, df.to_dict('records'))
  query = """Match (n)-[r]->(m)
          Return n,r.value,m"""
  result = session.run(query)
  example_graph = []
  for record in result.data():
    x = record['n']['id']
    y = record['r.value']
    z = record['m']['id']
    example_graph.append((x,y,z))
example_graph


[(1, 11, 2),
 (1, 11, 3),
 (2, 11, 3),
 (3, 11, 2),
 (3, 11, 4),
 (4, 11, 1),
 (4, 11, 2),
 (4, 11, 3),
 (4, 12, 5),
 (5, 12, 1),
 (5, 12, 2),
 (5, 12, 6)]

P2

Funciones auxiliares

In [336]:
def mix_tuples(tup):
    ((a_mod, b_mod, original_tuple), i) = tup
    return [
        ((a_mod, b_mod, i), original_tuple),
        ((i, a_mod, b_mod), original_tuple),
        ((b_mod, i, a_mod), original_tuple)
    ]

In [337]:
def triangulos(lista):
  tuplas = []
  for arista_pivote in lista:
    x = arista_pivote[0]
    y = arista_pivote[2]
    for arista in lista:
      if arista[0] == y:
        z = arista[2]
        for arista_final in lista:
          if arista_final[0] == z and arista_final[2] == x:
            triangulo = {x, y, z}
            if triangulo not in tuplas:
              tuplas.append(tuple(triangulo))
      return tuplas

In [338]:
def triangulos(lista):
  tuplas = []
  for arista_pivote in lista:
    x = arista_pivote[0]
    y = arista_pivote[2]
    for arista in lista:
      if arista[0] == y:
        z = arista[2]
        for arista_final in lista:
          if arista_final[0] == z and arista_final[2] == x:
            triangulo = {x, y, z}
            if triangulo not in tuplas:
              tuplas.append(tuple(triangulo))
  return tuplas

In [339]:
def cycles(b):
  tuples_rdd = sc.parallelize(example_graph)
  #citar
  indices_rdd = sc.parallelize(list(range(b)))
  tuples_rdd = tuples_rdd.map(lambda t: (t[0] % b, t[2] % b, t))
  cartesian_product = tuples_rdd.cartesian(indices_rdd)
  mixed_tuples_rdd = cartesian_product.flatMap(mix_tuples)
  mixed_tuples_rdd = mixed_tuples_rdd.distinct()
  mixed_tuples_rdd = mixed_tuples_rdd.groupByKey()
  mixed_tuples_rdd = mixed_tuples_rdd.mapValues(list)
  mixed_tuples_rdd = mixed_tuples_rdd.mapValues(triangulos)
  mixed_tuples_rdd = mixed_tuples_rdd.flatMap(lambda x: x[1])
  mixed_tuples_rdd = mixed_tuples_rdd.distinct()
  return mixed_tuples_rdd.collect()

cycles(2)

[(2, 3, 4), (1, 3, 4)]

P3

In [340]:
def cuadrados(lista):
  tuplas = []
  for arista_pivote in lista:
    x = arista_pivote[0]
    y = arista_pivote[2]
    for arista in lista:
      if arista[0] == y:
        z = arista[2]
        for arista_2 in lista:
          if arista_2[0] == z:
            w = arista_2[2]
            for arista_final in lista:
              if arista_final[0] == w and arista_final[2] == x:
                cuadrado = {x, y, z, w}
                if len(cuadrado) == 4:
                  cuadrado = sorted(list(cuadrado))
                  comb = sorted((arista_pivote,arista,arista_2,arista_final))
                  comb = set(comb)
                  if comb not in tuplas:
                    tuplas.append((tuple(cuadrado),comb))
  return tuplas

In [341]:
def flatten_tuples(record):
    first_tuple, second_tuple = record
    flattened_first_tuple = tuple(x for sub in first_tuple for x in (sub if isinstance(sub, tuple) else (sub,)))
    return (flattened_first_tuple, second_tuple)

In [342]:
def transformar(elemento):
    primera_tupla = elemento[0]
    lista_tuplas = list(elemento[1][0])
    return (primera_tupla, lista_tuplas)

In [343]:
def cycles_square(b):
  combinaciones = [(i, j) for i in range(b) for j in range(b)]
  tuples_rdd = sc.parallelize(example_graph)
  tuples_rdd = tuples_rdd.map(lambda t: (t[0] % b, t[2] % b, t))
  combinaciones = [(i, j) for i in range(b) for j in range(b)]
  cartesian_product = sc.parallelize([(var, comb) for var in tuples_rdd.collect() for comb in combinaciones])
  mixed_tuples_rdd = cartesian_product.flatMap(mix_tuples)
  mixed_tuples_rdd = mixed_tuples_rdd.map(flatten_tuples)
  mixed_tuples_rdd = mixed_tuples_rdd.distinct()
  mixed_tuples_rdd = mixed_tuples_rdd.groupByKey()
  mixed_tuples_rdd = mixed_tuples_rdd.mapValues(list)
  mixed_tuples_rdd = mixed_tuples_rdd.mapValues(cuadrados)
  mixed_tuples_rdd = mixed_tuples_rdd.flatMap(lambda x: x[1])
  mixed_tuples_rdd = mixed_tuples_rdd.groupByKey()
  mixed_tuples_rdd = mixed_tuples_rdd.mapValues(list)
  mixed_tuples_rdd = mixed_tuples_rdd.map(transformar)
  return mixed_tuples_rdd.collect()


In [344]:
def encontrar_cuadrados(A, L, M, b, rdd):
    cuadrados = cycles_square(b)
    d_l = dict()
    for i in range(len(L)):
        d_l[L[i]] = i
    for cuadrado in cuadrados:
      nodos = cuadrado[0]
      aristas = cuadrado[1]
      d_v = dict()
      for i in range(len(nodos)):
        d_v[nodos[i]] = i
      counter = 0
      for arista in aristas:
        if M[d_l[arista[1]]][d_v[arista[0]]][d_v[arista[2]]]:
          counter += 1
      if counter == 4:
        print(nodos, 'es cuadrado en M')


A = ["x", "y", "z", "w"]
L = [11, 12]

M = [
    [[0, 1, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 0]],
    [[0, 0, 0, 0],
    [0, 0, 0, 0],
    [0, 0, 0, 1],
    [1, 0, 0, 0]]
]
rdd = example_graph


encontrar_cuadrados(A,L,M,b,rdd)


(2, 3, 4, 5) es cuadrado en M
(1, 3, 4, 5) es cuadrado en M
